### ### final3 - 처리 사항
- 변수제거
> groupId,matchId,killplace,(killpoint, rankpoint, winpoint => Weighted),damagedealt,matchduration, numgroups, roadkills, teamkill,vehicle destroys, maxplace

- 이상치 제거
> matchType은 이상치 없음<br>
> 'assists_mean', 'boosts_mean','headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean','longestKill_mean', 'rideDistance_mean','swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean','DBNOs_mean','revives_mean' => quantile 상위 0.005퍼센트 이상치 제거<br>
> 평균값이기 때문에 walkingdistance==0이고 winplaceperc >=0.5 은 제거(핵쟁이일 가능성)

- 데이터 변환
> Headshotkills => headshotkillsrate = headshotkill/kill<br>
> matchId, groupId 별로 평균값으로 컬럼 대체<br>
> match 평균 변수 추가<br>
> matchsize 변수 추가<br>
> 각_mean 변수/ 각_match_mean 변수 => 컬럼개수 줄이고, 해당 판에서 그 사람의 위치를 비율로 만들어줄 수 있음

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm_notebook
import sklearn.preprocessing

In [2]:
# dataframe display 옵션
pd.set_option('display.max_columns' , 50)
pd.set_option('display.width', None)

In [3]:
df_train = pd.read_csv('./data/train_V2.csv', dtype={
                    'assists': 'int16',
                    'boosts': 'int16',
                    'DBNOs': 'int16',
                    'headshotKills': 'int16',
                    'heals': 'int16',
                    'kills': 'int16',
                    'killStreaks': 'int16',
                    'longestKill': 'float32',
                    'revives': 'int16',
                    'rideDistance': 'float32',
                    'swimDistance': 'float32',
                    'vehicleDestroys': 'int16',
                    'walkDistance': 'float32',
                    'weaponsAquired': 'int16',
                    'Weighted': 'float32',
                    'winPlacePerc': 'float32'
                })

In [4]:
# 필요 없는 칼럼 제거
del_list = ['maxPlace','damageDealt','killPlace', 'killPoints',
            'matchDuration','numGroups', 'rankPoints','roadKills','teamKills','vehicleDestroys','winPoints']
for col in del_list :
    del df_train[col]

In [42]:
# ## 메모리 절약
# def reduce_mem_usage(df):
#     """ iterate through all the columns of a dataframe and modify the data type
#         to reduce memory usage.        
#     """
#     #start_mem = df.memory_usage().sum() / 1024**2
#     #print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
#     for col in df.columns:
#         col_type = df[col].dtype

#         if col_type != object:
#             c_min = df[col].min()
#             c_max = df[col].max()
#             if str(col_type)[:3] == 'int':
#                 if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
#                     df[col] = df[col].astype(np.int8)
#                 elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
#                     df[col] = df[col].astype(np.int16)
#                 elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
#                     df[col] = df[col].astype(np.int32)
#                 elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
#                     df[col] = df[col].astype(np.int64)  
#             else:
#                 if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
#                     df[col] = df[col].astype(np.float32)
#                 else:
#                     df[col] = df[col].astype(np.float64)

#     #end_mem = df.memory_usage().sum() / 1024**2
#     #print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
#     #print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

#     return df

In [43]:
# df_train = reduce_mem_usage(df_train)

In [5]:
df_train_mean = df_train.groupby(['matchId','groupId']).mean()
df_train_size = df_train.groupby(['matchId']).size().reset_index(name='match_size')
df_train_match_mean = df_train.groupby(['matchId']).mean()

In [6]:
df_train = pd.merge(df_train, df_train_mean, suffixes=["","_mean"], how = 'left', on=['matchId','groupId'])

In [7]:
del df_train_mean

In [8]:
df_train = pd.merge(df_train,df_train_match_mean, suffixes=["","_match_mean"], how="left", on=['matchId'])

In [9]:
del df_train_match_mean

In [10]:
df_train = pd.merge(df_train, df_train_size, how='left', on=['matchId'])

In [11]:
del df_train_size

In [20]:
df_train_cvt = df_train.copy()

In [23]:
# 회귀분석을 하기 위해 문자열을 범주값으로 변환
matchTyp = {'squad-fpp': 3, 'duo': 2, 'solo-fpp': 1, 'squad': 3, 'duo-fpp': 2, 'solo': 1,
       'normal-squad-fpp': 4, 'crashfpp': 4, 'flaretpp': 4, 'normal-solo-fpp': 4,
       'flarefpp': 4, 'normal-duo-fpp': 4, 'normal-duo': 4, 'normal-squad': 4,
       'crashtpp': 4, 'normal-solo': 4 }

df_train_cvt['matchType'] = df_train_cvt['matchType'].replace(matchTyp)

In [24]:
#필요 없는 칼럼 제거
del_list = ['assists', 'boosts', 'DBNOs',
       'headshotKills', 'heals', 'kills', 'killStreaks', 'longestKill',
       'revives', 'rideDistance', 'swimDistance', 'walkDistance',
       'weaponsAcquired','winPlacePerc_match_mean','winPlacePerc_mean']
for col in del_list :
    del df_train_cvt[col]

In [25]:
df_train_cvt.columns

Index(['Id', 'groupId', 'matchId', 'matchType', 'winPlacePerc', 'assists_mean',
       'boosts_mean', 'DBNOs_mean', 'headshotKills_mean', 'heals_mean',
       'kills_mean', 'killStreaks_mean', 'longestKill_mean', 'revives_mean',
       'rideDistance_mean', 'swimDistance_mean', 'walkDistance_mean',
       'weaponsAcquired_mean', 'assists_match_mean', 'boosts_match_mean',
       'DBNOs_match_mean', 'headshotKills_match_mean', 'heals_match_mean',
       'kills_match_mean', 'killStreaks_match_mean', 'longestKill_match_mean',
       'revives_match_mean', 'rideDistance_match_mean',
       'swimDistance_match_mean', 'walkDistance_match_mean',
       'weaponsAcquired_match_mean', 'match_size'],
      dtype='object')

In [27]:
# null값 확인 및 
# inplace=False로 하면 기존 혹은 새로운 변수에 할당해야 하고, inplace = True 하면 해당변수에 적용됨
# na가 포함된 행을 제거하는 것은 데이터 소실이 크기 때문에 inplace=False가 default
df_train_cvt.dropna(inplace = True)
print(df_train_cvt.isnull().any().any())

False


In [61]:
solo_train_df = df_train_cvt[df_train_cvt['matchType']==1]

In [62]:
del solo_train_df['DBNOs_mean']
del solo_train_df['revives_mean']
del solo_train_df['DBNOs_match_mean']
del solo_train_df['revives_match_mean']

In [63]:
# solo
quantile_list = ['assists_mean', 'boosts_mean',
       'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean',
       'longestKill_mean', 'rideDistance_mean',
       'swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean']
quantile_dic = {}
for col in quantile_list:
    tmp_np = round(np.quantile(solo_train_df[col],0.995),5)
    quantile_dic[col] = tmp_np
print(quantile_dic)

{'assists_mean': 1.0, 'boosts_mean': 9.0, 'headshotKills_mean': 4.0, 'heals_mean': 14.0, 'kills_mean': 8.0, 'killStreaks_mean': 2.0, 'longestKill_mean': 266.70001, 'rideDistance_mean': 7904.485, 'swimDistance_mean': 218.89999, 'walkDistance_mean': 4613.485, 'weaponsAcquired_mean': 11.0}


In [64]:
# 이상치 제거 1
for col in quantile_dic:
    solo_train_df = solo_train_df[solo_train_df[col] <= quantile_dic[col]]

In [65]:
# 이상치 제거 2
solo_train_df.drop(index=solo_train_df[(solo_train_df['walkDistance_mean']==0)&(solo_train_df['winPlacePerc']>=0.5)].index,inplace=True)

In [66]:
# data 변환 - headshotkills
solo_train_df['headshotKillrate_mean'] = solo_train_df['headshotKills_mean']/solo_train_df['kills_mean']
solo_train_df['headshotKillrate_mean'].fillna(value=0,inplace=True)
solo_train_df['headshotKillrate_match_mean'] = solo_train_df['headshotKills_match_mean']/solo_train_df['kills_match_mean']
solo_train_df['headshotKillrate_match_mean'].fillna(value=0,inplace=True)

In [67]:
mean_list = ['assists_mean', 'boosts_mean', 'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean', 'longestKill_mean',
 'rideDistance_mean', 'swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean','headshotKillrate_mean']
mean_match_list = ['assists_match_mean',
 'boosts_match_mean','headshotKills_match_mean', 'heals_match_mean', 'kills_match_mean', 'killStreaks_match_mean', 'longestKill_match_mean',
 'rideDistance_match_mean', 'swimDistance_match_mean', 'walkDistance_match_mean', 'weaponsAcquired_match_mean','headshotKillrate_match_mean']
name_list = ['assists', 'boosts', 'headshotKills', 'heals', 'kills', 'killStreaks', 'longestKill',
 'rideDistance', 'swimDistance', 'walkDistance', 'weaponsAcquired','headshotKillrate']
print(len(mean_list))
print(len(mean_match_list))
print(len(name_list))

12
12
12


In [68]:
for i in range(len(mean_list)):
    mean = mean_list[i]
    match = mean_match_list[i]
    name = name_list[i]  
    solo_train_df[name+'_ratio'] = solo_train_df[mean]/(solo_train_df[match])
    solo_train_df[name+'_ratio'].fillna(value=0,inplace=True)
    del solo_train_df[mean]
    del solo_train_df[match]
round(solo_train_df,4)

In [76]:
solo_train_df.isna().any().any()

False

In [77]:
# winPlacePerc 컬럼위치 변경
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]
solo_train_df = change_column_order(solo_train_df,'winPlacePerc',len(list(solo_train_df.columns)))

In [78]:
solo_train_df.describe()

,matchType,match_size,assists_ratio,boosts_ratio,headshotKills_ratio,heals_ratio,kills_ratio,killStreaks_ratio,longestKill_ratio,rideDistance_ratio,swimDistance_ratio,walkDistance_ratio,weaponsAcquired_ratio,headshotKillrate_ratio,winPlacePerc
count,691048.0,691048.000000,691048.000000,691048.000000,691048.000000,691048.000000,691048.000000,691048.000000,691048.000000,691048.000000,691048.000000,691048.000000,691048.000000,691048.000000,691048.000000
mean,1.0,94.153982,0.865407,0.897580,0.870474,0.868676,0.896163,0.951509,0.874787,0.911003,0.714860,0.940224,0.974541,0.411668,0.472736
std,0.0,9.232758,4.314324,1.536483,2.197412,1.885820,1.385634,1.119863,1.765610,3.534308,4.784254,1.041599,0.621145,1.042220,0.293433
min,1.0,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.0,94.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.107070,0.522667,0.000000,0.218800
50%,1.0,96.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.494809,0.921519,0.000000,0.463900
75%,1.0,97.000000,0.000000,1.189873,0.000000,1.010204,1.111111,2.044444,0.905613,0.000000,0.000000,1.539298,1.350000,0.000000,0.723400
max,1.0,100.000000,100.000000,25.526316,28.000000,33.739130,10.378378,5.684211,19.827480,99.000000,100.000008,17.000000,4.163347,14.500000,1.000000


In [79]:
solo_train_df.to_csv('./data/final3/final3_solo_Data.csv',sep=',', encoding='utf-8', index=False)

### duo

In [80]:
duo_train_df = df_train_cvt[df_train_cvt['matchType']==2]

#  duo 리스트
quantile_list = ['assists_mean', 'boosts_mean',
       'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean',
       'longestKill_mean', 'rideDistance_mean',
       'swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean','DBNOs_mean','revives_mean']
quantile_dic = {}
for col in quantile_list:
    tmp_np = round(np.quantile(duo_train_df[col],0.995),5)
    quantile_dic[col] = tmp_np
print(quantile_dic)

# 이상치 제거 1
for col in quantile_dic:
    duo_train_df = duo_train_df[duo_train_df[col] <= quantile_dic[col]]
    
# 이상치 제거 2
duo_train_df.drop(index=duo_train_df[(duo_train_df['walkDistance_mean']==0)&(duo_train_df['winPlacePerc']>=0.5)].index,inplace=True)
 
# data 변환 - headshotkills
duo_train_df['headshotKillrate_mean'] = duo_train_df['headshotKills_mean']/duo_train_df['kills_mean']
duo_train_df['headshotKillrate_mean'].fillna(value=0,inplace=True)
duo_train_df['headshotKillrate_match_mean'] = duo_train_df['headshotKills_match_mean']/duo_train_df['kills_match_mean']
duo_train_df['headshotKillrate_match_mean'].fillna(value=0,inplace=True)

# data 변환 - ration
mean_list = ['assists_mean', 'boosts_mean', 'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean', 'longestKill_mean',
 'rideDistance_mean', 'swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean','headshotKillrate_mean','DBNOs_mean','revives_mean']
mean_match_list = ['assists_match_mean',
 'boosts_match_mean','headshotKills_match_mean', 'heals_match_mean', 'kills_match_mean', 'killStreaks_match_mean', 'longestKill_match_mean',
 'rideDistance_match_mean', 'swimDistance_match_mean', 'walkDistance_match_mean', 'weaponsAcquired_match_mean',
                   'headshotKillrate_match_mean','DBNOs_match_mean','revives_match_mean']
name_list = ['assists', 'boosts', 'headshotKills', 'heals', 'kills', 'killStreaks', 'longestKill',
 'rideDistance', 'swimDistance', 'walkDistance', 'weaponsAcquired','headshotKillrate','DBNOs','revives']
print(len(mean_list))
print(len(mean_match_list))
print(len(name_list))

for i in range(len(mean_list)):
    mean = mean_list[i]
    match = mean_match_list[i]
    name = name_list[i]  
    duo_train_df[name+'_ratio'] = duo_train_df[mean]/(duo_train_df[match])
    duo_train_df[name+'_ratio'].fillna(value=0,inplace=True)
    del duo_train_df[mean]
    del duo_train_df[match]
round(duo_train_df,4)

# winPlacePerc 컬럼위치 변경
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]
duo_train_df = change_column_order(duo_train_df,'winPlacePerc',len(list(duo_train_df.columns)))

duo_train_df.to_csv('./data/final3/final3_duo_Data.csv',sep=',', encoding='utf-8', index=False)

{'assists_mean': 2.0, 'boosts_mean': 7.5, 'headshotKills_mean': 2.5, 'heals_mean': 12.5, 'kills_mean': 6.5, 'killStreaks_mean': 2.0, 'longestKill_mean': 224.10001, 'rideDistance_mean': 7707.5, 'swimDistance_mean': 177.54999, 'walkDistance_mean': 4695.5, 'weaponsAcquired_mean': 9.5, 'DBNOs_mean': 4.0, 'revives_mean': 1.5}
14
14
14


### squad

In [81]:
squad_train_df = df_train_cvt[df_train_cvt['matchType']==3]

#  squad 리스트
quantile_list = ['assists_mean', 'boosts_mean',
       'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean',
       'longestKill_mean', 'rideDistance_mean',
       'swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean','DBNOs_mean','revives_mean']
quantile_dic = {}
for col in quantile_list:
    tmp_np = round(np.quantile(squad_train_df[col],0.995),5)
    quantile_dic[col] = tmp_np
print(quantile_dic)

# 이상치 제거 1
for col in quantile_dic:
    squad_train_df = squad_train_df[squad_train_df[col] <= quantile_dic[col]]
    
# 이상치 제거 2
squad_train_df.drop(index=squad_train_df[(squad_train_df['walkDistance_mean']==0)&(squad_train_df['winPlacePerc']>=0.5)].index,inplace=True)
 
# data 변환 - headshotkills
squad_train_df['headshotKillrate_mean'] = squad_train_df['headshotKills_mean']/squad_train_df['kills_mean']
squad_train_df['headshotKillrate_mean'].fillna(value=0,inplace=True)
squad_train_df['headshotKillrate_match_mean'] = squad_train_df['headshotKills_match_mean']/squad_train_df['kills_match_mean']
squad_train_df['headshotKillrate_match_mean'].fillna(value=0,inplace=True)

# data 변환 - ration
mean_list = ['assists_mean', 'boosts_mean', 'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean', 'longestKill_mean',
 'rideDistance_mean', 'swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean','headshotKillrate_mean','DBNOs_mean','revives_mean']
mean_match_list = ['assists_match_mean',
 'boosts_match_mean','headshotKills_match_mean', 'heals_match_mean', 'kills_match_mean', 'killStreaks_match_mean', 'longestKill_match_mean',
 'rideDistance_match_mean', 'swimDistance_match_mean', 'walkDistance_match_mean', 'weaponsAcquired_match_mean',
                   'headshotKillrate_match_mean','DBNOs_match_mean','revives_match_mean']
name_list = ['assists', 'boosts', 'headshotKills', 'heals', 'kills', 'killStreaks', 'longestKill',
 'rideDistance', 'swimDistance', 'walkDistance', 'weaponsAcquired','headshotKillrate','DBNOs','revives']
print(len(mean_list))
print(len(mean_match_list))
print(len(name_list))

for i in range(len(mean_list)):
    mean = mean_list[i]
    match = mean_match_list[i]
    name = name_list[i]  
    squad_train_df[name+'_ratio'] = squad_train_df[mean]/(squad_train_df[match])
    squad_train_df[name+'_ratio'].fillna(value=0,inplace=True)
    del squad_train_df[mean]
    del squad_train_df[match]
round(squad_train_df,4)

# winPlacePerc 컬럼위치 변경
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]
squad_train_df = change_column_order(squad_train_df,'winPlacePerc',len(list(squad_train_df.columns)))

squad_train_df.to_csv('./data/final3/final3_squad_Data.csv',sep=',', encoding='utf-8', index=False)

{'assists_mean': 2.33333, 'boosts_mean': 6.25, 'headshotKills_mean': 2.0, 'heals_mean': 11.5, 'kills_mean': 5.5, 'killStreaks_mean': 2.0, 'longestKill_mean': 205.52499, 'rideDistance_mean': 7558.3335, 'swimDistance_mean': 149.71483, 'walkDistance_mean': 4664.0, 'weaponsAcquired_mean': 9.0, 'DBNOs_mean': 4.5, 'revives_mean': 1.6}
14
14
14


### other

In [82]:
other_train_df = df_train_cvt[df_train_cvt['matchType']==4]

#  other 리스트
quantile_list = ['assists_mean', 'boosts_mean',
       'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean',
       'longestKill_mean', 'rideDistance_mean',
       'swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean','DBNOs_mean','revives_mean']
quantile_dic = {}
for col in quantile_list:
    tmp_np = round(np.quantile(other_train_df[col],0.995),5)
    quantile_dic[col] = tmp_np
print(quantile_dic)

# 이상치 제거 1
for col in quantile_dic:
    other_train_df = other_train_df[other_train_df[col] <= quantile_dic[col]]
    
# 이상치 제거 2
other_train_df.drop(index=other_train_df[(other_train_df['walkDistance_mean']==0)&(other_train_df['winPlacePerc']>=0.5)].index,inplace=True)
 
# data 변환 - headshotkills
other_train_df['headshotKillrate_mean'] = other_train_df['headshotKills_mean']/other_train_df['kills_mean']
other_train_df['headshotKillrate_mean'].fillna(value=0,inplace=True)
other_train_df['headshotKillrate_match_mean'] = other_train_df['headshotKills_match_mean']/other_train_df['kills_match_mean']
other_train_df['headshotKillrate_match_mean'].fillna(value=0,inplace=True)

# data 변환 - ration
mean_list = ['assists_mean', 'boosts_mean', 'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean', 'longestKill_mean',
 'rideDistance_mean', 'swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean','headshotKillrate_mean','DBNOs_mean','revives_mean']
mean_match_list = ['assists_match_mean',
 'boosts_match_mean','headshotKills_match_mean', 'heals_match_mean', 'kills_match_mean', 'killStreaks_match_mean', 'longestKill_match_mean',
 'rideDistance_match_mean', 'swimDistance_match_mean', 'walkDistance_match_mean', 'weaponsAcquired_match_mean',
                   'headshotKillrate_match_mean','DBNOs_match_mean','revives_match_mean']
name_list = ['assists', 'boosts', 'headshotKills', 'heals', 'kills', 'killStreaks', 'longestKill',
 'rideDistance', 'swimDistance', 'walkDistance', 'weaponsAcquired','headshotKillrate','DBNOs','revives']
print(len(mean_list))
print(len(mean_match_list))
print(len(name_list))

for i in range(len(mean_list)):
    mean = mean_list[i]
    match = mean_match_list[i]
    name = name_list[i]  
    other_train_df[name+'_ratio'] = other_train_df[mean]/(other_train_df[match])
    other_train_df[name+'_ratio'].fillna(value=0,inplace=True)
    del other_train_df[mean]
    del other_train_df[match]
round(other_train_df,4)

# winPlacePerc 컬럼위치 변경
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]
other_train_df = change_column_order(other_train_df,'winPlacePerc',len(list(other_train_df.columns)))

other_train_df.to_csv('./data/final3/final3_other_Data.csv',sep=',', encoding='utf-8', index=False)

{'assists_mean': 7.0, 'boosts_mean': 7.0, 'headshotKills_mean': 6.625, 'heals_mean': 14.0, 'kills_mean': 22.0, 'killStreaks_mean': 3.28571, 'longestKill_mean': 254.47185, 'rideDistance_mean': 9650.7775, 'swimDistance_mean': 180.576, 'walkDistance_mean': 4685.75, 'weaponsAcquired_mean': 43.33333, 'DBNOs_mean': 13.85714, 'revives_mean': 2.5}
14
14
14


In [83]:
duo_train_df.head(30)

,Id,groupId,matchId,matchType,match_size,assists_ratio,boosts_ratio,headshotKills_ratio,heals_ratio,kills_ratio,killStreaks_ratio,longestKill_ratio,rideDistance_ratio,swimDistance_ratio,walkDistance_ratio,weaponsAcquired_ratio,headshotKillrate_ratio,DBNOs_ratio,revives_ratio,winPlacePerc
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,2,98,2.450000,1.689655,0.000000,1.139535,1.076923,0.924528,0.324837,1.988448,0.000000,0.672988,0.630225,0.000000,0.942308,0.000000,0.7755
11,8de328a74658a9,f643df9df3877c,80170383d90003,2,95,0.000000,0.000000,0.000000,0.241730,0.000000,0.000000,0.000000,0.000000,0.000000,0.343244,0.705945,0.000000,0.586420,0.000000,0.4043
15,12d8d4bd94312c,fe52d481bae68b,6fd9e765ddd0c5,2,95,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.088208,1.013720,0.000000,1.507937,2.968750,0.2766
20,02ace8c6e58461,a4bc548028f800,80f2b8448e474b,2,95,0.000000,3.614130,2.968750,3.986014,2.111111,1.792453,0.657598,3.880678,0.000000,1.626500,1.413690,1.406250,1.637931,3.653846,0.6383
22,9b2961d4d51f91,799d0a4d61dc3c,e833ca2282169d,2,97,0.000000,0.475490,7.461538,0.765789,1.114943,0.881818,4.710496,1.807782,0.000000,0.980096,1.347222,6.692308,0.795082,0.000000,0.7500
23,0b6fbdfb59c994,7a75c3e86934f6,8b0a78c005cea0,2,97,2.852941,3.063158,3.880000,0.629870,4.796703,2.108696,6.234831,1.969267,0.000000,1.388441,1.019520,0.808889,4.619048,0.000000,0.9592
25,4c45dc732689ec,8e0a0ea95d3596,37f43ba55ec0a4,2,93,1.722222,3.206897,0.000000,2.715328,2.113636,2.846939,3.808192,0.993893,0.000000,1.995286,1.660714,0.000000,3.152542,0.000000,0.8696
27,dbf611495bfda3,10cbb86844dee0,2cdae31ee18601,2,94,0.000000,1.724771,4.947368,0.559524,3.241379,3.000000,1.024790,1.205105,0.000000,0.894971,1.260054,1.526316,2.517857,0.000000,0.7234
29,ac5b57ff39979c,857cc55b2b6001,e019e04dee4f19,2,92,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006229,0.090954,0.000000,0.000000,0.000000,0.0000
30,3a4e362d28f411,b4fbf11bf33ac3,cf3d655571b26b,2,94,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.337400,1.236842,0.000000,0.000000,0.000000,0.4255


In [84]:
duo_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1255949 entries, 2 to 4446958
Data columns (total 20 columns):
Id                        1255949 non-null object
groupId                   1255949 non-null object
matchId                   1255949 non-null object
matchType                 1255949 non-null int64
match_size                1255949 non-null int64
assists_ratio             1255949 non-null float64
boosts_ratio              1255949 non-null float64
headshotKills_ratio       1255949 non-null float64
heals_ratio               1255949 non-null float64
kills_ratio               1255949 non-null float64
killStreaks_ratio         1255949 non-null float64
longestKill_ratio         1255949 non-null float32
rideDistance_ratio        1255949 non-null float32
swimDistance_ratio        1255949 non-null float32
walkDistance_ratio        1255949 non-null float32
weaponsAcquired_ratio     1255949 non-null float64
headshotKillrate_ratio    1255949 non-null float64
DBNOs_ratio               1